# 5. Bring in Covid-19 Data for Business Insights

Now that we have some basic enterprise data ready, let's bring in some Covid-19 data.  We'll merge these two data sets together to try to answer a few questions related to our business:

- **Are there any regions where the Covid-19 case rate is rising week to week?**
- **Are any of those in my top 100 sales regions?**
- **Who are the affected salespeople so I can notify them?**

To get started, we'll load in case rate data from the New York Times into a Postgres table.  There's a public data set here:

https://s3-us-west-1.amazonaws.com/starschema.covid/NYT_US_COVID19.csv (<a href="https://github.com/nytimes/covid-19-data/blob/master/LICENSE">license</a>)

The NYT Covid-19 data also uses FIPS to designate the location of cases and deaths, so we can cross-reference that data with our sales data which is already indicated by FIPS.


## 5.1 Create the table for the NYT data

In [ ]:
from my_connect import my_connect

connection = my_connect()
cursor = connection.cursor()
q = "DROP TABLE IF EXISTS nyt_us_covid19;"
cursor.execute(q)
connection.commit()

q = """
CREATE TABLE IF NOT EXISTS nyt_us_covid19 (
                id SERIAL PRIMARY KEY,
                date DATE,
                county VARCHAR(200),
                state VARCHAR(100),
                fips VARCHAR(5),
                cases INTEGER,
                deaths INTEGER,
                iso3166_1 VARCHAR(10),
                iso3166_2 VARCHAR(10),
                cases_since_prev_day INTEGER,
                deaths_since_prev_day INTEGER,
                last_update_date TIMESTAMP,
                last_reported_flag BOOLEAN
               )
"""
cursor.execute(q)
connection.commit()

## 5.2 Download the NYT Covid-19 data file from S3

Download the CSV file from S3 and save a copy locally, referenced by CSV_FILE below:


In [ ]:
import requests
import os

SOURCE = "https://s3-us-west-1.amazonaws.com/starschema.covid/NYT_US_COVID19.csv"
CSV_TEMP = os.path.join(os.getcwd(), "nyt_latest.csv")
r = requests.get(SOURCE)
open(CSV_TEMP, 'wb').write(r.content)

## 5.3 Import the CSV into the nyt_us_covid19 table

In [ ]:
import psycopg2
import psycopg2.sql as sql
import pandas
import os

connection = my_connect()
cursor = connection.cursor()

# Zero out the table first
q = "DELETE FROM nyt_us_covid19;"
cursor.execute(q)
connection.commit()

CSV_TEMP = os.path.join(os.getcwd(), "nyt_latest.csv")

q2 = sql.SQL("""
COPY nyt_us_covid19(date, county, state, fips, cases, deaths, iso3166_1, iso3166_2,cases_since_prev_day, 
deaths_since_prev_day, last_update_date, last_reported_flag) FROM {} CSV HEADER;
""")

cursor.execute(q2.format(sql.Literal(CSV_TEMP)))
connection.commit()

os.unlink(CSV_TEMP)
    
df = pandas.io.sql.read_sql_query("SELECT * FROM nyt_us_covid19 ORDER BY date DESC LIMIT 5", connection)
print(df.head())

You should now have a table with all of the Covid-19 case data.  You can re-run this at any time to load the latest data.  Now let's do some preparation on our data to get it into a form we can use to apply it to our business questions.

Recall the first question we want to answer:
- **Are there any regions where the Covid-19 case rate is rising week to week?**

To get this, we can run a query on the data, giving us last week's and this week's case rate by FIPS code, along with the difference.  In the second query below, note the following clause:
```
INTO cases_change_by_fips
```

This creates a new table with the results.  This intermediate table will make it easier to join up with our enterprise sales data later on.

In these examples, we'll use the weeks of 5-April-2020 and 12-April-2020 as our 'current' weeks, even though by the time you read this they will no longer be current.

In [ ]:
connection = my_connect()
cursor = connection.cursor()

# This selects the cases for the week of April 5 in each FIPS region into a temp table
q1 = """
DROP TABLE IF EXISTS temp_table1;
SELECT fips, 
SUM(CASE WHEN date BETWEEN '2020-04-05' AND '2020-04-11' AND fips <> 'None' THEN cases_since_prev_day ELSE 0.00 END) AS week1
INTO TEMP TABLE temp_table1
FROM nyt_us_covid19
GROUP BY (fips);
"""

# This selects the cases for the subsequent week (April 12) and computes the difference
q2 = """
DROP TABLE IF EXISTS cases_change_by_fips;
SELECT nyt_us_covid19.fips, week1,
SUM(CASE WHEN date BETWEEN '2020-04-12' AND '2020-04-18' AND nyt_us_covid19.fips <> 'None' 
    THEN cases_since_prev_day ELSE 0.00 END) - week1 AS case_change,
-- This prevents division by zero when there are no new cases
(CASE WHEN week1 = 0 THEN 0 ELSE 
  (SUM(CASE WHEN date BETWEEN '2020-04-12' AND '2020-04-18' AND nyt_us_covid19.fips <> 'None' 
    THEN cases_since_prev_day ELSE 0.00 END) - week1) / week1
END) AS percent_change
INTO cases_change_by_fips
FROM nyt_us_covid19 
JOIN temp_table1 ON (temp_table1.fips = nyt_us_covid19.fips)
GROUP BY (nyt_us_covid19.fips, week1)
ORDER BY case_change desc;
"""

cursor.execute(q1)
connection.commit()
cursor.execute(q2)
connection.commit()

## 5.4 Check the results

Let's look at the top FIPS regions where the percent change from week to week was greater than 20% and the case count was at least 100:

In [ ]:
import pandas

connection = my_connect()

q = """
SELECT *
FROM cases_change_by_fips 
WHERE percent_change > .20 AND week1 > 100
ORDER BY case_change desc
"""
df = pandas.io.sql.read_sql_query(q, connection)
print(df.head())

Here's what the output should look like:

```        
    fips   week1  case_change  percent_change
0  25017  2404.0       1021.0        0.424709
1  44007   787.0        949.0        1.205845
2  25009  1496.0        522.0        0.348930
3  46099   357.0        481.0        1.347339
4  06065   766.0        405.0        0.528721
```

### Data quality concern: spot-check your query results

Let's spot-check some of the data to see if the query looks right.  Since 25017 is at the top of the list, let's look at it:

In [ ]:
df = pandas.io.sql.read_sql_query("SELECT date, cases_since_prev_day FROM nyt_us_covid19  \
WHERE date BETWEEN '2020-04-05' AND '2020-04-11' AND fips = '25017'", connection)
print(df)
print(df.sum())
df = pandas.io.sql.read_sql_query("SELECT date, cases_since_prev_day FROM nyt_us_covid19 \
WHERE date BETWEEN '2020-04-12' AND '2020-04-18' AND fips = '25017'", connection)
print(df)
print(df.sum())


Here's what the output should look like:

```
         date  cases_since_prev_day
0  2020-04-05                   164
1  2020-04-06                   318
2  2020-04-07                   237
3  2020-04-08                   358
4  2020-04-09                   500
5  2020-04-10                   402
6  2020-04-11                   425
cases_since_prev_day    2404
dtype: int64
         date  cases_since_prev_day
0  2020-04-12                   788
1  2020-04-13                   323
2  2020-04-14                   271
3  2020-04-15                   427
4  2020-04-16                   525
5  2020-04-17                   538
6  2020-04-18                   553
cases_since_prev_day    3425
dtype: int64
```
Looks good!  We had 2404 cases the first week, and 3425 the second week, a difference of 1021 cases, or (1021/2404 = .4247), which is what the first row of our table above says it should be.

## 5.5 This answers our first business question:

- **Are there any regions where the Covid-19 case rate is rising week to week?**

The answer is yes, and we know exactly which regions are increasing by how much.  Now let's look at our second question:

- **Are any of those in my top 100 sales regions?**

We have all the data we need, we just need to put it together.  We'll join the case increase data we just created with the sales data by FIPS code.  Since we only care about significant case rate increases, we'll limit our search only to those regions where the percent_change is greater than 20% (0.2).

To make it easier to understand, we'll create some intermediate tables to hold results, then we'll join them together at the end.

The first table we need is the sales data summarized by FIPS.  For that, we'll create a table called total_sales_by_fips.  Here's the schema and the query:

## 5.6 Create a total_sales_by_fips table for the top 100 sales regions:

In [ ]:
# Note: on my machine, this SELECT...INTO query will sometimes hang in Postgres 10.  Restarting the service fixes it.

from my_connect import my_connect
import pandas

connection = my_connect()
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS total_sales_by_fips;")
connection.commit()

# Select all sales data summarized by each FIPS code for the top 100 FIPS
q = """
SELECT SUM(sales.amount) AS total_sales, fips 
INTO total_sales_by_fips
FROM sales
GROUP BY (fips)
ORDER BY total_sales DESC LIMIT 100;
"""
cursor.execute(q)
connection.commit()

In [ ]:
# Verify we have data
df = pandas.io.sql.read_sql_query("SELECT * FROM total_sales_by_fips", connection)
print(df.head())

## 5.7 Join the case change rate data with the top-100 regions data.

Now all we need to do is join the cases_change_by_fips data with the total_sales_by_fips data and see if there's any intersection.  We can use an INNER JOIN for this, which will only generate a result where the FIPS matches between both.  Since our total_sales_by_fips table only has the top 100 FIPS regions we care about, we'll get a list of those top regions that have a concerning case rate increase.

In [ ]:
q = """
SELECT total_sales, cases_change_by_fips.fips, week1 AS first_week_cases, case_change, percent_change 
FROM total_sales_by_fips
INNER JOIN cases_change_by_fips ON total_sales_by_fips.fips = cases_change_by_fips.fips
WHERE cases_change_by_fips.percent_change > .20 AND cases_change_by_fips.week1 > 100
ORDER BY cases_change_by_fips.percent_change DESC;
"""
connection = my_connect()
df = pandas.io.sql.read_sql_query(q, connection)
print(df.head(100))

This is what the output should look like:
```
    total_sales   fips  first_week_cases  case_change  percent_change
0     211503.33  27053             212.0        199.0        0.938679
1     166378.42  13089             309.0        251.0        0.812298
2     208426.75  06081             114.0         72.0        0.631579
3     195980.34  48439             322.0        202.0        0.627329
4     201578.86  13067             252.0        147.0        0.583333
5     170168.17  48453             260.0        142.0        0.546154
6     190369.15  06065             766.0        405.0        0.528721
7     219742.09  06029             164.0         86.0        0.524390
8     216130.19  39049             337.0        172.0        0.510386
9     214920.82  25017            2404.0       1021.0        0.424709
10    197162.99  48141             173.0         63.0        0.364162
11    191036.56  12103             104.0         37.0        0.355769
12    190502.24  25009            1496.0        522.0        0.348930
13    202892.51  13135             281.0         97.0        0.345196
14    224533.30  47157             441.0        142.0        0.321995
15    179627.83  08031             413.0        128.0        0.309927
16    187676.02  17043             458.0        136.0        0.296943
17    178294.47  13121             487.0        132.0        0.271047
18    194325.48  25027             907.0        223.0        0.245865
19    214895.50  09003            1106.0        258.0        0.233273
20    181039.78  51059             559.0        128.0        0.228980
21    171498.17  12057             201.0         44.0        0.218905
```

## 5.8 Business insight:

Remember our second business-related question:

- **Are any of those in my top 100 sales regions?**

Out of the top 100 counties we sell product in, 21 of them have had a significant increase in the number of cases.  That is a concern, and we should notify the relevant salespeople so they can react accordingly, hence our third question:

- **Who are the affected salespeople so I can notify them?**

We'll join the salesperson data based on the sate in which the FIPS region resides.  We'll start with the query above and modify it.

In [ ]:
q = """
DROP TABLE IF EXISTS final_report;
SELECT total_sales, 
       cases_change_by_fips.fips, 
       fips.area_name, 
       fips.state, 
       percent_change, 
       salesperson.name
INTO final_report
FROM total_sales_by_fips
INNER JOIN cases_change_by_fips ON total_sales_by_fips.fips = cases_change_by_fips.fips
INNER JOIN fips ON fips.fipstxt = cases_change_by_fips.fips
INNER JOIN salesperson ON salesperson.state = fips.state
WHERE cases_change_by_fips.percent_change > .20 and cases_change_by_fips.week1 > 100
ORDER BY name;
"""
connection = my_connect()
cursor = connection.cursor()
cursor.execute(q)
connection.commit()


In [ ]:
from IPython.display import display

q2 = """
SELECT total_sales AS "Total Sales", 
       fips AS "FIPS", 
       area_name AS "County", 
       state AS "State", 
       percent_change AS "Increase", 
       name AS "Salesperson Name"
FROM final_report
"""
connection = my_connect()
df = pandas.io.sql.read_sql_query(q2, connection)

# Pandas dataframe number formatting and inline bar chart example
df2 = df.style.bar(subset=['Increase'], align='mid', color=['#FF8888'])\
        .format({'Total Sales': "{:,.2f}", 'Increase': "{:.0%}"})
# Nicer display inside a Jupyter notebook
display(df2)

## ...and there you have it.

We now have a list of the affected salespeople, and we've given them enough information that they can take whatever action they think is needed -- changing inventory levels, contacting stores in the region, checking for any local lockdown orders that might affect sales, etc.

### Sales Regions: Covid-19 Weekly Case Increases of > 20% 
<img src="images/final-report.png" align="left">



## Visualization: mapping the case rate increases

You can use a tool like PowerBI to geographically map the case rate change data.  See <a href="https://docs.microsoft.com/en-us/power-bi/visuals/desktop-shape-map">this link</a> for more details on how to enable Shape Maps in PowerBI.  Since our case rate data contains FIPS locations, Shape Maps can map it directly.  Here's an example using the ```final_report``` table; I had to work with the settings and filtering for a while to get what I wanted, but this should give you a sense of what's possible.  

<img src="images/map-percent-change-2.png">

Other tools such as Tableau also have this capability.

# Next notebook: next steps, automating, etc.

<a href="6. Next Steps.ipynb">Go to the next notebook -&gt;</a>


*Contents © Copyright 2020 HP Development Company, L.P. SPDX-License-Identifier: MIT*